# Loan Prediction 3

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
set(train).difference(test)

{'Loan_Status'}

In [4]:
train['data'] = 'train'
test['Loan_Status'] = np.nan
test['data'] = 'test'

In [5]:
df = pd.concat([train, test], axis=0)
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,data
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y,train
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,train
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,train
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,train
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,train


In [6]:
df.isnull().sum()

Loan_ID                0
Gender                24
Married                3
Dependents            25
Education              0
Self_Employed         55
ApplicantIncome        0
CoapplicantIncome      0
LoanAmount            27
Loan_Amount_Term      20
Credit_History        79
Property_Area          0
Loan_Status          367
data                   0
dtype: int64

In [7]:
list(zip(df.columns, df.dtypes, df.nunique()))

[('Loan_ID', dtype('O'), 981),
 ('Gender', dtype('O'), 2),
 ('Married', dtype('O'), 2),
 ('Dependents', dtype('O'), 4),
 ('Education', dtype('O'), 2),
 ('Self_Employed', dtype('O'), 2),
 ('ApplicantIncome', dtype('int64'), 752),
 ('CoapplicantIncome', dtype('float64'), 437),
 ('LoanAmount', dtype('float64'), 232),
 ('Loan_Amount_Term', dtype('float64'), 12),
 ('Credit_History', dtype('float64'), 2),
 ('Property_Area', dtype('O'), 3),
 ('Loan_Status', dtype('O'), 2),
 ('data', dtype('O'), 2)]

In [8]:
df['Gender'].fillna('Male', inplace=True)
df['Married'].fillna('Yes', inplace=True)
df['Dependents'].fillna('0', inplace=True)
df['Self_Employed'].fillna('No', inplace=True)
df['LoanAmount'].fillna(142.6, inplace=True)
df['Loan_Amount_Term'].fillna(360.0, inplace=True)
df['Credit_History'].fillna(1.0, inplace=True)

In [9]:
df.isnull().sum()

Loan_ID                0
Gender                 0
Married                0
Dependents             0
Education              0
Self_Employed          0
ApplicantIncome        0
CoapplicantIncome      0
LoanAmount             0
Loan_Amount_Term       0
Credit_History         0
Property_Area          0
Loan_Status          367
data                   0
dtype: int64

In [10]:
pd.crosstab(df['Loan_Status'], df['Loan_Amount_Term'], dropna=False)

Loan_Amount_Term,6.0,12.0,36.0,60.0,84.0,120.0,180.0,240.0,300.0,350.0,360.0,480.0
Loan_Status,,,,,,,,,,,,
N,0,0,2,0,1,0,15,1,5,0,159,9
Y,0,1,0,2,3,3,29,3,8,0,367,6


In [11]:
df['Loan_Amount_Term'] = np.where(df['Loan_Amount_Term'].isin([6.0,12.0, 36.0, 60.0, 84.0, 120.0, 240.0,350.0]), 'LoanAmtTerm1', df['Loan_Amount_Term'])
df['Loan_Amount_Term'] = np.where(df['Loan_Amount_Term'] == '360.0', 'LoanAmtTerm2', df['Loan_Amount_Term'])
df['Loan_Amount_Term'] = np.where(df['Loan_Amount_Term'].isin(['300.0', '480.0']), 'LoanAmtTerm3', df['Loan_Amount_Term'])
df['Loan_Amount_Term'] = np.where(df['Loan_Amount_Term'] == '180.0', 'LoanAmtTerm4', df['Loan_Amount_Term'])

In [12]:
df['Loan_Amount_Term'].value_counts(dropna=False)

LoanAmtTerm2    843
LoanAmtTerm4     66
LoanAmtTerm3     43
LoanAmtTerm1     29
Name: Loan_Amount_Term, dtype: int64

In [13]:
del df['Loan_ID']
cat_cols = df.select_dtypes('object').columns
cat_cols = cat_cols[:-2]
cat_cols

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'Loan_Amount_Term', 'Property_Area'],
      dtype='object')

In [14]:
df = pd.get_dummies(df, columns=cat_cols, prefix=cat_cols, drop_first=True)
df['Loan_Status'] = np.where(df['Loan_Status'] == 'Y', 1, 0)
df.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Credit_History,Loan_Status,data,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Loan_Amount_Term_LoanAmtTerm2,Loan_Amount_Term_LoanAmtTerm3,Loan_Amount_Term_LoanAmtTerm4,Property_Area_Semiurban,Property_Area_Urban
0,5849,0.0,142.6,1.0,1,train,1,0,0,0,0,0,0,1,0,0,0,1
1,4583,1508.0,128.0,1.0,0,train,1,1,1,0,0,0,0,1,0,0,0,0
2,3000,0.0,66.0,1.0,1,train,1,1,0,0,0,0,1,1,0,0,0,1
3,2583,2358.0,120.0,1.0,1,train,1,1,0,0,0,1,0,1,0,0,0,1
4,6000,0.0,141.0,1.0,1,train,1,0,0,0,0,0,0,1,0,0,0,1


In [15]:
train = df[df['data'] == 'train']
test = df[df['data'] == 'test']
del train['data']
test.drop(['Loan_Status', 'data'], axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [47]:
X = train.drop('Loan_Status', axis=1)
y = train['Loan_Status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

In [48]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8943089430894309

In [51]:
model.fit(X, y)
test_pred = model.predict(test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [52]:
Submission = pd.read_csv('sample_submission.csv')
del Submission['Loan_Status']
Submission['Loan_Status'] = test_pred
Submission['Loan_Status'] = np.where(Submission['Loan_Status'] == 1, "Y", 'N')
Submission.tail()

,Loan_ID,Loan_Status
362,LP002971,Y
363,LP002975,Y
364,LP002980,Y
365,LP002986,Y
366,LP002989,Y


In [53]:
Submission.to_csv('ss.csv', index=False)

# Model Building

In [37]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, train_test_split

In [17]:
train_sub, test_sub = train_test_split(train, test_size=0.2, random_state=19)

In [36]:
x_train = train_sub.drop('Loan_Status', axis=1)
y_train = train_sub['Loan_Status']
x_test = test_sub.drop('Loan_Status', axis=1)
y_test = test_sub['Loan_Status']

In [40]:
from sklearn.linear_model import LogisticRegression
logr = LogisticRegression().fit(x_train, y_train)
y_pred = logr.predict(x_test)
accuracy_score(y_pred, y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8943089430894309

In [19]:
clf = DecisionTreeClassifier()

In [20]:
params = { 'class_weight':[None], 
        'criterion':['entropy'],
        'max_depth':[6],
        'min_samples_leaf':[18],
        'min_samples_split':[35]}

In [21]:
random = RandomizedSearchCV(clf,
                           param_distributions=params,
                           cv = 10,
                           scoring = 'accuracy',
                           verbose = 2,
                           n_iter = 10,
                           random_state=42)

In [22]:
random.fit(x_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None 
[CV]  min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None, total=   0.0s
[CV] min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None 
[CV]  min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None, total=   0.0s
[CV] min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None 
[CV]  min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None, total=   0.0s
[CV] min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None 
[CV]  min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None, total=   0.0s
[CV] min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_w

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=DecisionTreeClassifier(class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort=False,
                                                    random_state=None,
                                                    splitter='best'

In [23]:
random.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=18, min_samples_split=35,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [24]:
random.cv_results_

{'mean_fit_time': array([0.01760068]),
 'std_fit_time': array([0.00888573]),
 'mean_score_time': array([0.00279884]),
 'std_score_time': array([0.00312302]),
 'param_min_samples_split': masked_array(data=[35],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_min_samples_leaf': masked_array(data=[18],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[6],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_criterion': masked_array(data=['entropy'],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_class_weight': masked_array(data=[None],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'min_samples_split': 35,
   'min_samples_leaf': 18,
   'max_depth': 6,
   'criterion': 'entropy',
   'class_weight': None}],
 'split0_test_score': array([0.76]),
 'split1_test_score':

In [25]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [26]:
report(random.cv_results_,5)

Model with rank: 1
Mean validation score: 0.754 (std: 0.06297)
Parameters: {'min_samples_split': 35, 'min_samples_leaf': 18, 'max_depth': 6, 'criterion': 'entropy', 'class_weight': None}



In [27]:
dtree=random.best_estimator_

In [43]:
dtree.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=18, min_samples_split=35,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [44]:
y_pred = dtree.predict(x_test)
accuracy_score(y_pred, y_test)

0.8455284552845529

In [29]:
dotfile = open("mytree.dot", 'w')

tree.export_graphviz(dtree,out_file=dotfile,
                     feature_names=x_train.columns,
                    class_names=["0","1"],
                     proportion=True)
dotfile.close()

In [41]:
y_pred = random.predict_proba(x_test)[:,1]
y_pred

array([0.        , 0.68627451, 0.64285714, 0.88888889, 0.94444444,
       0.17391304, 0.04545455, 0.625     , 0.68627451, 0.625     ,
       0.75      , 0.68627451, 0.68627451, 0.        , 0.72222222,
       0.68627451, 0.68627451, 0.5       , 0.94444444, 0.64285714,
       0.42105263, 0.88888889, 0.17391304, 1.        , 1.        ,
       0.17391304, 0.68627451, 0.68627451, 0.72222222, 0.94444444,
       0.97368421, 0.17391304, 0.17391304, 0.79166667, 0.72222222,
       0.97368421, 0.17391304, 0.79166667, 0.97368421, 0.86206897,
       0.94444444, 0.64285714, 0.68627451, 0.5       , 0.        ,
       0.75      , 0.97368421, 0.625     , 0.68627451, 0.86206897,
       0.64285714, 0.64285714, 0.79166667, 0.68627451, 1.        ,
       0.97368421, 0.72222222, 0.        , 1.        , 0.68627451,
       0.17391304, 0.97368421, 0.68627451, 0.79166667, 0.88888889,
       0.68627451, 0.97368421, 0.79166667, 0.97368421, 0.17391304,
       0.75      , 0.64285714, 0.72222222, 0.94444444, 0.17391

In [31]:
cutoffs = np.linspace(0.01,0.99,99)

real = y_test
ks_all = []
acc_all = []
f1_all = []

for cutoff in cutoffs:
    predicted = (y_pred>cutoff).astype(int)
    TP = ((predicted == 1) & (real == 1)).sum()
    FP = ((predicted == 1) & (real == 0)).sum()
    TN = ((predicted == 0) & (real == 0)).sum()
    FN = ((predicted == 0) & (real == 1)).sum()
    
    P = TP+FN
    N = TN+FP
    
    Pr = (TP/(TP+FP))
    Re = (TP/(FN+TP))
    
    KS = (TP/P)-(FP/N)
    acc = (TP+TN)/(P+N)
    f1 = (2*Pr*Re)/(Pr+Re)
    
    ks_all.append(KS)
    acc_all.append(acc)
    f1_all.append(f1)

In [32]:
max(ks_all)

0.6402597402597403

In [33]:
max(acc_all)

0.8861788617886179

In [34]:
max(f1_all)

0.9247311827956989

In [35]:
my_cutoff = cutoffs[acc_all == max(acc_all)][0]
my_cutoff

0.18000000000000002

In [279]:
y_predprob = (y_pred>my_cutoff).astype(int)
y_predprob

array([0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1])

In [280]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test, y_predprob)

array([[23, 12],
       [ 2, 86]], dtype=int64)

In [281]:
accuracy_score(y_test, y_predprob)

0.8861788617886179

In [282]:
X = train.drop('Loan_Status', axis=1)
Y = train['Loan_Status']

In [283]:
random.fit(X, Y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None 
[CV]  min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None, total=   0.0s
[CV] min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None 
[CV]  min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None, total=   0.0s
[CV] min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None 
[CV]  min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None, total=   0.0s
[CV] min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None 
[CV]  min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_weight=None, total=   0.0s
[CV] min_samples_split=35, min_samples_leaf=18, max_depth=6, criterion=entropy, class_w

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=DecisionTreeClassifier(class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort=False,
                                                    random_state=None,
                                                    splitter='best'

In [284]:
random.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=18, min_samples_split=35,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [285]:
y_pred = random.predict_proba(test)[:,1]
y_pred

array([0.80165289, 0.84615385, 0.80165289, 1.        , 0.66666667,
       0.94444444, 0.7826087 , 0.22727273, 0.80165289, 0.89655172,
       0.66666667, 0.55555556, 0.55172414, 0.22727273, 0.89655172,
       0.80165289, 0.59090909, 1.        , 0.59090909, 0.89655172,
       0.5862069 , 0.66666667, 0.55172414, 0.80165289, 0.59090909,
       0.04761905, 0.80165289, 0.59090909, 1.        , 1.        ,
       0.7826087 , 0.80165289, 0.55555556, 1.        , 0.84615385,
       0.22727273, 0.59090909, 1.        , 0.80165289, 0.80165289,
       0.55172414, 0.80165289, 0.94444444, 0.80645161, 0.55172414,
       0.59090909, 0.80165289, 0.89655172, 0.59090909, 0.55555556,
       0.84615385, 1.        , 0.80645161, 0.55172414, 0.55172414,
       0.22727273, 0.80165289, 1.        , 0.04761905, 0.88888889,
       0.94444444, 0.80165289, 0.55555556, 0.        , 0.80165289,
       1.        , 0.22727273, 0.22727273, 0.55555556, 0.04761905,
       0.7826087 , 0.7826087 , 0.80165289, 0.59090909, 0.88888

In [286]:
trainscore = random.predict_proba(X)[:,1]    # this gives prob of predicted response on the training data
real = Y                                    # The actual hardclass response

In [287]:
cutoffs = np.linspace(0.01,0.99,99)

real = Y 
ks_all = []
acc_all = []
f1_all = []

for cutoff in cutoffs:
    predicted = (trainscore>cutoff).astype(int)
    TP = ((predicted == 1) & (real == 1)).sum()
    FP = ((predicted == 1) & (real == 0)).sum()
    TN = ((predicted == 0) & (real == 0)).sum()
    FN = ((predicted == 0) & (real == 1)).sum()
    
    P = TP+FN
    N = TN+FP
    
    Pr = (TP/(TP+FP))
    Re = (TP/(FN+TP))
    
    KS = (TP/P)-(FP/N)
    acc = (TP+TN)/(P+N)
    f1 = (2*Pr*Re)/(Pr+Re)
    
    ks_all.append(KS)
    acc_all.append(acc)
    f1_all.append(f1)

In [288]:
max(ks_all)

0.5173035150078988

In [289]:
max(acc_all)

0.8094462540716613

In [290]:
my_cutoff = cutoffs[acc_all == max(acc_all)][0]
my_cutoff

0.23

In [295]:
Submission = pd.read_csv('sample_submission.csv')
del Submission['Loan_Status']
Submission['Loan_Status'] = (y_pred>my_cutoff).astype(int)
Submission['Loan_Status'] = np.where(Submission['Loan_Status'] == 1, "Y", 'N')
Submission.tail()

,Loan_ID,Loan_Status
362,LP002971,Y
363,LP002975,Y
364,LP002980,Y
365,LP002986,Y
366,LP002989,Y


In [296]:
Submission.to_csv('DT_Sub.csv', index=False)

In [297]:
Submission['Loan_Status'].value_counts()

Y    308
N     59
Name: Loan_Status, dtype: int64